In [62]:
!pip install py_arkworks_bls12381

from py_arkworks_bls12381 import G1Point, G2Point, GT, Scalar
from sympy import symbols, div, Poly, mod_inverse
import random
from random import randint
from numpy.polynomial import polynomial as P
import numpy as np

In [63]:
# Tools

def assert_integer_polynomial(f):
    assert all(int(coeff) == coeff for coeff in f)

def polynomial_division_mod_p(dividend, divisor, p):
    x = symbols('x')

    A = Poly(dividend, x).set_modulus(p)
    B = Poly(divisor, x).set_modulus(p)

    Q, R = div(A, B)
    return Q.all_coeffs(), R.all_coeffs()

# evaluate polynomial f on point s
# f = [3, 2, 1] <-> 3*x^2 + 2*x + 1
def polynomial_eval(f, s):
    sum = f[-1]
    for i in range(1, len(f)):
        sum += f[-(i + 1)] * s**i
    return sum

def trapdoor_polynomial_eval(group_identity, f, powers_of_tau):
    c_f = group_identity

    for i in range(len(f)):
        f_i = int(f[-(i + 1)])
        if f_i > 0:
            c_f += powers_of_tau[i] * Scalar(f_i)
        else:
            c_f -= powers_of_tau[i] * Scalar(-f_i)

    return c_f

In [64]:
# Initialize the Elliptic Curve
g = G1Point()
h = G2Point()
gt_gen = GT()

g_identity = G1Point.identity()
h_identity = G2Point.identity()

field_order = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
subgroup_order = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001

In [65]:
class Trusted_Setup:
    def __init__(self, d):
        self.d = d

    # input d, the maximum degree of a polynomial we will commit to
    # returns (ck, vk)
    # ck is the commitment key which is the list: ([1]_1, [tau]_1, ..., [tau ^ d]_1)
    # vk is the verification key which is a pair: ([1]_2, [tau]_2)
    def generate_public_parameters(self):
        keys = {}
        keys["ck"] = [];
        keys["vk"] = [];

        tau = Scalar(randint(0, 2**255 - 1))

        coef = Scalar(1)
        for _ in range(self.d):
            keys["ck"].append(g * coef)
            coef *= tau

        keys["vk"].append(h)
        keys["vk"].append(h * tau)

        return keys

In [66]:
class Prover:
    def __init__(self, ck, f):
        self.ck = ck
        self.f = f

    # input f is the polynomial we are commiting to
    # input ck is a list. The commitment key generated by the trusted setup
    # returns c_f, the commited point on G1
    def commit(self):
        assert len(self.f) <= len(self.ck), "Polynomial degree exceeds commitment key size"

        assert_integer_polynomial(self.f)

        return trapdoor_polynomial_eval(g_identity, self.f, self.ck)

    # input s is the point on G1
    def eval(self, s):
        z = int(polynomial_eval(self.f, s)) % subgroup_order

        numerator_poly = self.f.copy()
        numerator_poly[-1] -= z

        denominator_poly = [1, -s]

        q, _ = polynomial_division_mod_p(numerator_poly, denominator_poly,
                                         subgroup_order)
        return trapdoor_polynomial_eval(g_identity, q, ck)

In [67]:
class Verifier:
    def __init__(self, vk):
        self.vk = vk

    # c_f: commitment to the polynomial
    # evaluation of f on s is equal to z: f(s) = z
    # c_q commitment to the derived polynomial
    def verify(self, c_f, s, z, c_q):
        assert(self.vk[0] == h)

        lhs = GT.pairing(c_f - g * Scalar(z), self.vk[0])
        rhs = GT.pairing(c_q, self.vk[1] - h * Scalar(s))

        return lhs == rhs

In [68]:
# Test

max_d = 100
trusted_setup = Trusted_Setup(max_d)
pp = trusted_setup.generate_public_parameters()
ck = pp["ck"]
vk = pp["vk"]

def manual_polynomial_correct_commitment():
    # 3x^2 + 2x + 1
    f = [3, 2, 1]
    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()

    point = 2
    correct_evaluation = polynomial_eval(f, point)

    quotient_polynomial_commitment = prover.eval(point)

    return verifier.verify(polynomial_commitment, point, correct_evaluation,
                           quotient_polynomial_commitment)

def small_random_polynomial_wrong_evaluation_commitment():
    f_deg = 7
    max_coef = 7

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()

    point = randint(0, 100)

    correct_evaluation = int(polynomial_eval(f, point))
    wrong_evaluation = correct_evaluation + 1

    quotient_polynomial_commitment = prover.eval(point)

    return verifier.verify(polynomial_commitment, point, wrong_evaluation,
                           quotient_polynomial_commitment)

def small_random_polynomial_correct_commitment():
    f_deg = 100
    max_coef = 100

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()

    point = randint(0, 100)
    correct_evaluation = int(polynomial_eval(f, point))

    quotient_polynomial_commitment = prover.eval(point)

    return verifier.verify(polynomial_commitment, point, correct_evaluation,
                           quotient_polynomial_commitment)

assert(manual_polynomial_correct_commitment() == True)
assert(small_random_polynomial_wrong_evaluation_commitment() == False)
assert(small_random_polynomial_correct_commitment() == True)